In [3]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.applications import VGG16
from keras.models import Model
from keras.layers import GlobalAveragePooling2D
from xgboost import XGBClassifier

In [9]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [10]:
# Direktori dataset
dataset_dir = '/content/drive/MyDrive/PeriodontalDataset/periodontal_disease_dataset'
train_dir = f'{dataset_dir}/train'
val_dir = f'{dataset_dir}/val'

In [14]:
# Preprocessing dan augmentasi gambar untuk training dan validasi
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2
)

# Preprocessing gambar untuk testing
test_datagen = ImageDataGenerator(rescale=1./255)

# Generator gambar untuk training
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    subset='training'
)

# Generator gambar untuk validasi
validation_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    subset='validation'
)

# Generator gambar untuk testing
test_generator = test_datagen.flow_from_directory(
    val_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)

Found 106 images belonging to 2 classes.
Found 26 images belonging to 2 classes.
Found 44 images belonging to 2 classes.


In [16]:
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))


58889256/58889256 [==============================] - 0s 0us/step


In [17]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
model = Model(inputs=base_model.input, outputs=x)

In [18]:
for layer in base_model.layers:
    layer.trainable = False

In [21]:
# # Extract features for training and validation sets
# train_features = model.predict(train_dir)
# val_features = model.predict(val_dir)

# # Get labels
# train_labels = train_dir.classes
# val_labels = val_dir.classes

# Extract features for training and validation sets
train_features = model.predict(train_generator)
val_features = model.predict(validation_generator)

# Get labels
# Assuming your generators have a way to access the true labels
train_labels = train_generator.classes
val_labels = validation_generator.classes

1/1 [==============================] - 15s 15s/step


In [22]:
# Fit XGBoost model
xgb_model = XGBClassifier()

In [23]:
# Cross-validation
scores = cross_val_score(xgb_model, train_features, train_labels, cv=5)
print("Cross-Validation Accuracy: %.2f%%" % (scores.mean() * 100))

Cross-Validation Accuracy: 57.66%


In [24]:
# Train the final model
xgb_model.fit(train_features, train_labels)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [25]:
# Evaluate on validation set
val_predictions = xgb_model.predict(val_features)
accuracy = np.mean(val_predictions == val_labels)
print(f"Validation Accuracy: {accuracy * 100:.2f}%")

Validation Accuracy: 61.54%
